# Exporting and Importing MLRun Functions

In [1]:
# nuclio: ignore
import nuclio

## Write a function

In [2]:
import os
import zipfile

def open_archive(context, 
                 target_dir='content',
                 archive_url=''):
    """Open a file/object archive into a target directory"""
        
    # Define locations
    os.makedirs(target_dir, exist_ok=True)
    context.logger.info('Verified directories')
    
    # Extract dataset from zip
    context.logger.info('Extracting zip')
    zip_ref = zipfile.ZipFile(archive_url, 'r')
    zip_ref.extractall(target_dir)
    zip_ref.close()
    
    context.logger.info(f'extracted archive to {target_dir}')
    context.log_artifact('content', target_path=target_dir)


In [4]:
# nuclio: end-code

## Export to a file

In [3]:
import mlrun

In [4]:
# create job function object from notebook code
fn = mlrun.code_to_function('file_utils', runtime='job', 
                            handler=open_archive, image='mlrun/mlrun:latest')

In [19]:
print(fn.to_yaml())

kind: job
metadata:
  name: file_utils
spec:
  command: ''
  args: []
  image: mlrun/mlrun:latest
  volumes: []
  volume_mounts: []
  env: []
  description: file utilities
  build:
    functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlciBvbiAyMDE5LTEwLTI4IDIyOjAzCgppbXBvcnQgb3MKaW1wb3J0IHppcGZpbGUKCmRlZiBvcGVuX2FyY2hpdmUoY29udGV4dCwgCiAgICAgICAgICAgICAgICAgdGFyZ2V0X2Rpcj0nY29udGVudCcsCiAgICAgICAgICAgICAgICAgYXJjaGl2ZV91cmw9JycpOgogICAgIiIiT3BlbiBhIGZpbGUvb2JqZWN0IGFyY2hpdmUgaW50byBhIHRhcmdldCBkaXJlY3RvcnkiIiIKICAgICAgICAKICAgIG9zLm1ha2VkaXJzKHRhcmdldF9kaXIsIGV4aXN0X29rPVRydWUpCiAgICBjb250ZXh0LmxvZ2dlci5pbmZvKCdWZXJpZmllZCBkaXJlY3RvcmllcycpCiAgICAKICAgIGNvbnRleHQubG9nZ2VyLmluZm8oJ0V4dHJhY3RpbmcgemlwJykKICAgIHppcF9yZWYgPSB6aXBmaWxlLlppcEZpbGUoYXJjaGl2ZV91cmwsICdyJykKICAgIHppcF9yZWYuZXh0cmFjdGFsbCh0YXJnZXRfZGlyKQogICAgemlwX3JlZi5jbG9zZSgpCiAgICAKICAgIGNvbnRleHQubG9nZ2VyLmluZm8oZidleHRyYWN0ZWQgYXJjaGl2ZSB0byB7dGFyZ2V0X2Rpcn0nKQogICAgY29udGV4dC5sb2dfYXJ0aWZhY3QoJ2N

In [25]:
# save to a file (and can be pushed to a git)
fn.export('function.yaml')

## Import the function from Github and run

In [26]:
mlrun.mlconf.dbpath = 'http://mlrun-db:8080'

In [27]:
# load function from Github
xfn = mlrun.import_function('https://raw.githubusercontent.com/mlrun/functions/master/fileutils/function.yaml')

# configute it: mount on iguazio fabric, set as interactive (return stdout)
xfn.apply(mlrun.mount_v3io())
xfn.interactive = True

# create and run the task
images_path = '/User/mlrun/examples/images'
open_archive_task = mlrun.NewTask('download', handler='open_archive', 
               params={'target_dir': images_path},
               inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'})

# run
run = xfn.run(open_archive_task)

[mlrun] 2019-10-28 23:05:51,415 starting run download uid=699bbcb4cf7d44efb32393d329341d26  -> http://mlrun-db:8080
[mlrun] 2019-10-28 23:05:51,454 using in-cluster config.
[mlrun] 2019-10-28 23:05:51,484 Pod download-zrxvx created


/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


..
[mlrun] 2019-10-28 23:05:54,873 starting run download uid=699bbcb4cf7d44efb32393d329341d26  -> http://mlrun-db:8080
[mlrun] 2019-10-28 23:05:54,950 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2019-10-28 23:05:56,595 Verified directories
[mlrun] 2019-10-28 23:05:56,595 Extracting zip
[mlrun] 2019-10-28 23:06:16,188 extracted archive to /User/mlrun/examples/images
type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 699bbcb4cf7d44efb32393d329341d26 
[mlrun] 2019-10-28 23:06:16,215 run executed, status=completed


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...341d26,0,Oct 28 23:05:54,completed,download,commit=ed71f24a8066db6bb02cd7bdf7b9cf93209b3ec4host=download-zrxvxkind=localowner=iguaziorepo=https://github.com/mlrun/mlrun.git,archive_url,target_dir=/User/mlrun/examples/images,,content


type result.show() to see detailed results/progress or use CLI:
!mlrun get run --uid 699bbcb4cf7d44efb32393d329341d26 
[mlrun] 2019-10-28 23:06:21,260 run executed, status=completed
